In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from collections import Counter
import pandas as pd
import os
import glob
import copy
from pathlib import Path

from pprint import pprint
import itertools as it
from collections import Counter
import numpy as np

from aiida import load_profile
from aiida.orm.nodes.data.structure import StructureData

from aiida_quantumespresso.tools.pwinputparser import create_builder_from_file
from aiida_quantumespresso.calculations.pw import PwCalculation
from aiida_quantumespresso.calculations.pwimmigrant import PwimmigrantCalculation
from aiida.engine import submit
from aiida_quantumespresso.calculations.pw import PwCalculation
from aiida_quantumespresso.workflows.pw.relax import PwRelaxWorkChain
from aiida_quantumespresso.workflows.pw.base import PwBaseWorkChain
from aiida.orm import load_node, load_code

from qe_tools.exceptions import ParsingError

# from own_utils.calc_df import init_project_df

from pymatgen.io.ase import AseAtomsAdaptor

# import pybat
# from pybat import Cathode, LiRichCathode, Dimer, DimerNEBAnalysis

from ase.visualize import view
from ase.io.vasp import read_vasp, write_vasp
from ase.build.tools import sort

# from itables import init_notebook_mode
# init_notebook_mode(all_interactive=True)
pd.set_option('display.max_colwidth', None)

In [2]:
load_profile()

Profile<uuid='4fe00b7a32994cfca8b6cf43c2d22a53' name='jgeiger'>

In [3]:
# iurii_olivine_dir = '/home/jgeiger/projects/bat_uv_ml/data/olivines_iurii/LixMnPO4/Li1.00/DFT_plus_U/1_vcrelax/'
# iurii_olivine_file = 'LiMnPO4.vcrelax.1.in'
# iurii_pseudo_dir = '/home/jgeiger/projects/bat_uv_ml/data/olivines_iurii/Pseudopotentials/SSSP'
# iurri_spinel_dir = '/home/jgeiger/projects/bat_uv_ml/data/spinel_iurii/LixMn2O4/DFT+U/LiMn2O4/1_vcrelax'
# iurii_spinel_file = 'LiMn2O4.vcrelax.1.in'

In [4]:
spinel_dir_iurii = '/home/jgeiger/projects/bat_uv_ml/data/spinel_iurii'
olivine_dir_iurii = '/home/jgeiger/projects/bat_uv_ml/data/olivine_iurii'
project_dir = '/home/jgeiger/projects/bat_uv_ml'
pseudo_dir = '/usr/share/espresso/pseudo/'

In [5]:
def init_project_df(calc_dir, file_glob):
    # TODO: Extent for multiple file types, also for output files -> can be done by passing a tuple, which endswith accepts.
    # path_list = [os.path.dirname(path) for path in Path(calc_dir).rglob(file_glob)]
    path_list = []
    for root, dirs, files in os.walk(calc_dir):
        for file in files:
            if file.endswith(file_glob):
                path_list.append(os.path.join(root, file))
    # path_list = [_.replace('+', '\+') for _ in path_list]

    full_project_df = pd.DataFrame()
    full_project_df['abs_path'] = [os.path.dirname(_) for _ in path_list]
    full_project_df['calc_in'] = [os.path.basename(_) for _ in path_list]
    full_project_df['abs_path_in'] = path_list
    # ! Rel path buggy because it ends up being 
    # full_project_df['rel_path'] = full_project_df['abs_path'].str.replace(project_dir, '.')

    return full_project_df

def add_calcdata(df_in):
    calctype_regex = r"\.(pdos|dos|nscf|scf|hp|vcrelax)\."
    df_out = df_in.copy(deep=True)
    df_out['calc_name'] = df_out['calc_in'].apply(lambda x: Path(x).stem)
    df_out['calc_type'] = df_out['calc_in'].str.extract(calctype_regex)
    df_out['calc_out'] = df_out['calc_in'].str.replace('.in', '.out', regex=False)
    return df_out

# ! Also matches paremeters.in file, which results in calctype = nan
spinel_df_iurii = init_project_df(
    calc_dir=spinel_dir_iurii,
    file_glob='.in' # ('.in', '.out')
    )

olivine_df_iurii = init_project_df(
    calc_dir=olivine_dir_iurii,
    file_glob='.in'
    )
# olivine_df_iurii = olivine_df_iurii.dropna()


In [6]:
iurii_full_data_df = pd.concat([spinel_df_iurii, olivine_df_iurii], ignore_index=True)
iurii_full_data_df = add_calcdata(iurii_full_data_df)

iurii_full_data_df.shape
iurii_full_data_df.head()

# pandas write to csv
iurii_full_data_df.to_csv(os.path.join(project_dir, 'data', 'iurii_full_data_df.csv'), index=False)


(1304, 6)

,abs_path,calc_in,abs_path_in,calc_name,calc_type,calc_out
0,/home/jgeiger/projects/bat_uv_ml/data/spinel_iurii_unmod/LixMn1.5Ni0.5O4/DFT+U/LiMn1.5Ni0.5O4/5_pdos,LiMn1.5Ni0.5O4.scf.1.in,/home/jgeiger/projects/bat_uv_ml/data/spinel_iurii_unmod/LixMn1.5Ni0.5O4/DFT+U/LiMn1.5Ni0.5O4/5_pdos/LiMn1.5Ni0.5O4.scf.1.in,LiMn1.5Ni0.5O4.scf.1,scf,LiMn1.5Ni0.5O4.scf.1.out
1,/home/jgeiger/projects/bat_uv_ml/data/spinel_iurii_unmod/LixMn1.5Ni0.5O4/DFT+U/LiMn1.5Ni0.5O4/5_pdos,LiMn1.5Ni0.5O4.scf.2.in,/home/jgeiger/projects/bat_uv_ml/data/spinel_iurii_unmod/LixMn1.5Ni0.5O4/DFT+U/LiMn1.5Ni0.5O4/5_pdos/LiMn1.5Ni0.5O4.scf.2.in,LiMn1.5Ni0.5O4.scf.2,scf,LiMn1.5Ni0.5O4.scf.2.out
2,/home/jgeiger/projects/bat_uv_ml/data/spinel_iurii_unmod/LixMn1.5Ni0.5O4/DFT+U/LiMn1.5Ni0.5O4/5_pdos,LiMn1.5Ni0.5O4.dos.in,/home/jgeiger/projects/bat_uv_ml/data/spinel_iurii_unmod/LixMn1.5Ni0.5O4/DFT+U/LiMn1.5Ni0.5O4/5_pdos/LiMn1.5Ni0.5O4.dos.in,LiMn1.5Ni0.5O4.dos,dos,LiMn1.5Ni0.5O4.dos.out
3,/home/jgeiger/projects/bat_uv_ml/data/spinel_iurii_unmod/LixMn1.5Ni0.5O4/DFT+U/LiMn1.5Ni0.5O4/5_pdos,LiMn1.5Ni0.5O4.pdos.in,/home/jgeiger/projects/bat_uv_ml/data/spinel_iurii_unmod/LixMn1.5Ni0.5O4/DFT+U/LiMn1.5Ni0.5O4/5_pdos/LiMn1.5Ni0.5O4.pdos.in,LiMn1.5Ni0.5O4.pdos,pdos,LiMn1.5Ni0.5O4.pdos.out
4,/home/jgeiger/projects/bat_uv_ml/data/spinel_iurii_unmod/LixMn1.5Ni0.5O4/DFT+U/LiMn1.5Ni0.5O4/3_vcrelax,LiMn1.5Ni0.5O4.vcrelax.in,/home/jgeiger/projects/bat_uv_ml/data/spinel_iurii_unmod/LixMn1.5Ni0.5O4/DFT+U/LiMn1.5Ni0.5O4/3_vcrelax/LiMn1.5Ni0.5O4.vcrelax.in,LiMn1.5Ni0.5O4.vcrelax,vcrelax,LiMn1.5Ni0.5O4.vcrelax.out
